## 1. 모듈 불러오기

In [7]:
import pandas as pd
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

## 2. 설정

In [8]:
url_target = 'http://anthony.sogang.ac.kr/KoreaReview/'

In [9]:
nums = ['One','Two','Three','Four','Five']
vols = []
sechs = ['KoreaReviewVolume6FullText.html']
for i in nums:
    vols.append(str('KoreaReviewFullTextVolume'+i+'.html'))
post = vols+sechs

In [10]:
'''
pome, article을 제외한 나머지는 아래 문헌에서 참조.
pome, article은 임의로 넣은 분류
참고 : 이유정, 호머 헐버트의 한국 민속연구와 영역 설화집으로서의 "THE KOREA REVIEW"
'''
chapter = ['poem','article','odds and ends', 'question and answer','editorial comment',
          'news calendar', 'correspondence', 'review', 'korean history']

In [11]:
# html 저장
def shtml(url_target):
    browser.get(url_target)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [12]:
# 분류 함수
def classify(ind):
    raw = list()
    key = ['Volume','Number','Section','Title','Text']
    sec = '' # 임의값
    for word in ind:
        # Volume
        vol = 1
        # Number
        if word in number:
            num = word
        # Section
        if str.lower(word[-4:])=='poem':
            sec = 'poem'
        elif str.lower(word) in chapter:
            sec = str.lower(word)
        elif 'history of korea' in str.lower(word):
            sec = 'korean history'
        elif sec == 'odds and ends':
            sec = sec
        else: sec = 'article'
        # title
        ttl = str.lower(word).strip()
        while ttl[-1]=='.':
            ttl = ttl[:-1]
        # text
        txt = ''
        # sum
        val = [vol, num, sec, ttl, txt]
        dic = dict()
        for i in range(len(key)):
            dic[key[i]] = val[i]
        raw.append(dic)
    return pd.DataFrame(raw)

In [63]:
# 분류 함수(권호만)
def classify_s(ind):
    raw = list()
    key = ['Volume','Number','Text']
    sec = '' # 임의값
    for word in ind:
        # Volume
        vol = 1
        # Number
        if word in number:
            num = word
        # text
        txt = ''
        # sum
        val = [vol, num, txt]
        dic = dict()
        for i in range(len(key)):
            dic[key[i]] = val[i]
        raw.append(dic)
    return pd.DataFrame(raw).drop_duplicates()

In [13]:
# 번호 찾기
def textpoint(soup, number, slct):
    n = 0
    for i in tqdm(range(len(soup.select(slct)))):
        if 'THEKOREAREVIEW' in re.sub('[^A-Z]','',soup.select(slct)[i].text.replace('\n',' ')):
            n+=1
        if n==number:
            break
    return i

In [14]:
# raws = []

# browser=webdriver.Chrome('D:/py/chromedriver/chromedriver.exe')
# browser.get(url_target)

# for i in tqdm(range(len(post))):
#     browser.get(url_target+post[i])
#     html = browser.page_source
#     soup = BeautifulSoup(html, 'html.parser')
#     raws.append({'Vol':i+1, 'doc':soup.select('p.MsoNormal')})

In [15]:
browser=webdriver.Chrome('D:/py/chromedriver/chromedriver.exe')

<ipython-input-15-13ec56c2aa1e>:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser=webdriver.Chrome('D:/py/chromedriver/chromedriver.exe')


## 3. 각 년도 별 인덱스 및 크롤링
- 6개 년도 페이지의 형식이 모두 제각각이기 때문에 따로 작업을 진행

### 1) vol 1. (1901)

In [16]:
soup = shtml(url_target+post[0])

In [17]:
ind = list()
for i in tqdm(range(11,203)):
    if re.sub('\s+',' ',soup.select('p.MsoNormal')[i].text.split('\xa0')[0]) != '':
        ind.append(re.sub('\s+',' ',soup.select('p.MsoNormal')[i].text.split('\xa0')[0]))

100%|████████████████████████████████████████████████████████████████████████████████| 192/192 [00:07<00:00, 26.99it/s]


In [19]:
#분류
df = classify(ind)

# index 후처리
exc = ['Geo. Heber Jones','Jas. S. Gale.','F. H. Morsel.','C. C. Vinton. M. D.',
      'Rev. F. S. Miller', 'George C. Foulk', 'Rev. C. T. Collyer']
for i in range(len(exc)):
    exc[i] = str.lower(exc[i])
months = number.copy()
for i in range(len(months)):
    months[i] = str.lower(months[i])
df['Title'][100] = 'the ni-t’u'
for i in range(len(df)):
    if (df['Title'][i] in exc)|(df['Title'][i] == 'ancient korea')|(df['Title'][i] == 'introductory note'):
        df.drop(i, inplace=True)
    elif (df['Title'][i] in months)|(df['Title'][i] == 'odds and ends'):
        df.drop(i, inplace=True)
        
# 월 추가
months = []
for word in df['Number']:
    months.append(re.findall(r'\(([^)]+)', word)[0])
df['months']=months

<ipython-input-19-c09d6a209fba>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Title'][100] = 'the ni-t’u'


In [21]:
starting = textpoint(soup, 2, 'p.MsoNormal')

  8%|██████▍                                                                        | 316/3880 [00:05<01:04, 55.56it/s]


In [22]:
re.sub('\s+',' ',soup.select('p.MsoNormal')[321].text)

'The Spirit of the Bell.'

- 권호만 분류

In [70]:
raw_s = list()

# 텍스트 저장
for i in tqdm(range(starting,len(soup.select(slct)))):
    raw_s.append(re.sub('\s+',' ',soup.select(slct)[i].text).strip())

100%|██████████████████████████████████████████████████████████████████████████████| 3564/3564 [01:08<00:00, 52.06it/s]


In [82]:
df_s = classify_s(ind)

# 월 추가
months = []
for word in df_s['Number']:
    months.append(re.findall(r'\(([^)]+)', word)[0])
df_s['months']=months

In [83]:
df_s

,Volume,Number,Text,months
0,1,No. 1 (January),,January
18,1,No. 2 (February),,February
37,1,No. 3 (March),,March
52,1,No. 4 (April),,April
66,1,No. 5 (May),,May
83,1,No. 6 (June),,June
99,1,No. 7 (July),,July
111,1,No. 8 (August),,August
125,1,No. 9 (September),,September
141,1,No. 10 (October),,October


In [221]:
slct = 'p.MsoNormal'
ttl = '' # 초기값
para = list() # 초기값
mon = df['months'].unique()
        
# 텍스트 분류
for text in tqdm(raw_s):
    text.split()
    if (len(text)>=4)&(len(text)<=50):
        try: 
            if re.sub(',','',text).strip().split()[-2] in mon:
                txt = '\n '.join(para)
                df_s.loc[(df_s['months']==month),'Text'] = txt
                del txt
                para = list()
                month = re.sub(',','',text).strip().split()[-2]
        except: pass
    para.append(text)
txt = '\n '.join(para)
df_s.loc[(df_s['months']==month),'Text'] = txt

# 인덱스 초기화
df_s = df_s.reset_index().iloc[:,1:]

# 페이지 표시 제거
for i in tqdm(range(len(df_s))):
    for j in range(0,1000):
        df_s['Text'][i] = df_s['Text'][i].replace('[page {}]'.format(j+1),'')

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]<ipython-input-221-03b9603a6c18>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_s['Text'][i] = df_s['Text'][i].replace('[page {}]'.format(j+1),'')
100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:06<00:00,  1.94it/s]


In [231]:
df1 = df_s.copy()
df1

,Volume,Number,Text,months
0,1,No. 1 (January),"Volume 1, January 1901\n \n \n The Spirit of t...",January
1,1,No. 2 (February),February 1901\n \n The Opening Lines of Chang-...,February
2,1,No. 3 (March),March 1901\n \n Xylographic Art in Korea.\n \n...,March
3,1,No. 4 (April),"THE KOREA REVIEW, April 1901\n \n A Vagary of...",April
4,1,No. 5 (May),"THE KOREA REVIEW, May 1901\n \n A Vagary of F...",May
5,1,No. 6 (June),June 1901\n \n The Image of Gautama.\n A Chant...,June
6,1,No. 7 (July),"25 July, 1901.\n \n \n Editorial Comment.\n \n...",July
7,1,No. 8 (August),"THE KOREA REVIEW, August 1901\n \n The Korea B...",August
8,1,No. 9 (September),"THE KOREA REVIEW, September 1901\n \n The Seou...",September
9,1,No. 10 (October),"THE KOREA REVIEW, October 1901.\n \n A Notable...",October


In [229]:
months = list(df_s.months)

### 2) vol 2. (1902)

In [224]:
soup = shtml(url_target+post[1])

In [237]:
ind = list()
for i in tqdm(range(len(soup.select('p.PreformattedText')))):
    if re.sub('\s+',' ',soup.select('p.PreformattedText')[i].text.split('\xa0')[0]) != '':
        ind.append(re.sub('\s+',' ',soup.select('p.PreformattedText')[i].text.split('\xa0')[0]))

100%|██████████████████████████████████████████████████████████████████████████████| 4547/4547 [03:58<00:00, 19.06it/s]


In [270]:
# indexing
contents = []
for i in range(len(soup.select('p.MsoToc1'))):
    contents.append(' '.join(re.sub('\s+',' ',soup.select('p.MsoToc1')[i].text).split()[:2]))
contents

['JANUARY 1902.',
 'FEBRUARY 1902.',
 'MARCH 1902.',
 'APRIL 1902.',
 'MAY 1902.',
 'JUNE, 1902.',
 'JULY, 1902.',
 'AUGUST, 1902.',
 'SEPTEMBER 1902.',
 'OCTOBER, 1902.',
 'NOVEMBER, 1903.',
 'DECEMBER 1903.']

In [326]:
df2 = df_s.copy().iloc[:,:2]
df2['Volume'] = 2
df2['Text'] = ''
df2['Contents'] = contents
df2

,Volume,Number,Text,Contents
0,2,No. 1 (January),,JANUARY 1902.
1,2,No. 2 (February),,FEBRUARY 1902.
2,2,No. 3 (March),,MARCH 1902.
3,2,No. 4 (April),,APRIL 1902.
4,2,No. 5 (May),,MAY 1902.
5,2,No. 6 (June),,"JUNE, 1902."
6,2,No. 7 (July),,"JULY, 1902."
7,2,No. 8 (August),,"AUGUST, 1902."
8,2,No. 9 (September),,SEPTEMBER 1902.
9,2,No. 10 (October),,"OCTOBER, 1902."


In [332]:
raw = ind
slct = 'p.PreformattedText'
ttl = '' # 초기값
para = list() # 초기값
        
# 텍스트 분류
for text in tqdm(raw):
    ts = text
    if len(ts) >=2:
        if ts[-1]!='.':
            ts+='.'
    if ts in contents:
        txt = '\n '.join(para)
        df2.loc[(df2['Contents']==ttl),'Text'] = txt
        ttl = ts
        del txt
        para = list()
    para.append(text)
df2.loc[(df2['Contents']==ttl),'Text'] = '\n '.join(para)

100%|██████████████████████████████████████████████████████████████████████████| 2905/2905 [00:00<00:00, 171334.50it/s]


In [397]:
# 페이지 표시 제거
for i in tqdm(range(len(df2))):
    for j in range(0,1000):
        df2['Text'][i] = df2['Text'][i].replace('[page {}]'.format(j+1),'')

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]<ipython-input-397-844a429d2d17>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Text'][i] = df2['Text'][i].replace('[page {}]'.format(j+1),'')
100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:07<00:00,  1.56it/s]


In [398]:
df2

,Volume,Number,Text,Contents
0,2,No. 1 (January),JANUARY 1902.\n The status of Woman in Korea.\...,JANUARY 1902.
1,2,No. 2 (February),FEBRUARY 1902\n The Products of Korea.\n It is...,FEBRUARY 1902.
2,2,No. 3 (March),MARCH 1902\n The Status of Woman in Korea.\n I...,MARCH 1902.
3,2,No. 4 (April),APRIL 1902.\n A Submarine Adventure.\n The “Mo...,APRIL 1902.
4,2,No. 5 (May),"MAY 1902\n Necessity, the Mother of Invention....",MAY 1902.
5,2,No. 6 (June),"JUNE, 1902.\n Burial Customs.\n In a short ser...","JUNE, 1902."
6,2,No. 7 (July),"JULY, 1902\n Korean Fiction.\n A few weeks ago...","JULY, 1902."
7,2,No. 8 (August),"AUGUST, 1902\n Korean Currency.\n The history ...","AUGUST, 1902."
8,2,No. 9 (September),SEPTEMBER 1902\n The Treasures of Kyong-ju .\n...,SEPTEMBER 1902.
9,2,No. 10 (October),"OCTOBER, 1902\n The Korean Language.\n The Kor...","OCTOBER, 1902."


### 3) vol 3. (1903)

In [336]:
soup = shtml(url_target+post[2])

In [337]:
slct = 'p.MsoNormal'
starting = textpoint(soup, 2, slct)

# 텍스트 저장
raw = list()
for i in tqdm(range(starting,len(soup.select(slct)))):
    _ = re.sub('\s+',' ',soup.select(slct)[i].text).strip()
    if _ != '':
        raw.append(re.sub('\s+',' ',soup.select(slct)[i].text).strip())

100%|██████████████████████████████████████████████████████████████████████████████| 4520/4520 [03:47<00:00, 19.86it/s]


In [338]:
df3 = df_s.copy().iloc[:,:2]
df3['Volume'] = 3
df3['Text'] = ''
df3['months'] = months
df3

,Volume,Number,Text,months
0,3,No. 1 (January),,January
1,3,No. 2 (February),,February
2,3,No. 3 (March),,March
3,3,No. 4 (April),,April
4,3,No. 5 (May),,May
5,3,No. 6 (June),,June
6,3,No. 7 (July),,July
7,3,No. 8 (August),,August
8,3,No. 9 (September),,September
9,3,No. 10 (October),,October


In [342]:
slct = 'p.MsoNormal'
ttl = '' # 초기값
para = list() # 초기값
mon = df['months'].unique()

# 텍스트 분류
for text in tqdm(raw):
    text.split()
    if (len(text)>=4)&(len(text)<=25):
        try: 
            if re.sub(',','',text).strip().split()[-2] in mon:
                txt = '\n '.join(para)
                df3.loc[(df_s['months']==month),'Text'] = txt
                del txt
                para = list()
                month = re.sub(',','',text).strip().split()[-2]
        except: pass
    para.append(text)
txt = '\n '.join(para)
df3.loc[(df3['months']==month),'Text'] = txt

# 인덱스 초기화
df3 = df3.reset_index().iloc[:,1:]

# 페이지 표시 제거
for i in tqdm(range(len(df3))):
    for j in range(0,1000):
        df3['Text'][i] = df3['Text'][i].replace('[page {}]'.format(j+1),'')

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]<ipython-input-342-c1d6f2c65703>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Text'][i] = df3['Text'][i].replace('[page {}]'.format(j+1),'')
100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:07<00:00,  1.52it/s]


In [400]:
df3

,Volume,Number,Text,months
0,3,No. 1 (January),"Volume 3, January 1903\n A Notable Paper on Se...",January
1,3,No. 2 (February),"Volume 3, February 1903\n The Korean New Year ...",February
2,3,No. 3 (March),"Volume 3, March 1903\n The Test of Friendship....",March
3,3,No. 4 (April),"Volume 3, April 1903\n The Korea Mudang and Pa...",April
4,3,No. 5 (May),"Volume 3, May 1903\n The Privileges of the Cap...",May
5,3,No. 6 (June),"Volume 3, June 1903\n Note on Ch’oe Ch’i-Wun 2...",June
6,3,No. 7 (July),"Volume 3, July 1903.\n Korean and Formosan. 28...",July
7,3,No. 8 (August),"Volume 3, August 1903.\n The Peddlars’ Guild 3...",August
8,3,No. 9 (September),"Volume 3, September 1903\n Mudang and Pansu 38...",September
9,3,No. 10 (October),"Volume 3, October 1903.\n A Korean Poem. F. S....",October


### 4) vol 4. (1904)

In [344]:
soup = shtml(url_target+post[3])

In [345]:
slct = 'p.MsoNormal'
starting = textpoint(soup, 2, slct)

# 텍스트 저장
raw = list()
for i in tqdm(range(starting,len(soup.select(slct)))):
    _ = re.sub('\s+',' ',soup.select(slct)[i].text).strip()
    if _ != '':
        raw.append(re.sub('\s+',' ',soup.select(slct)[i].text).strip())

100%|██████████████████████████████████████████████████████████████████████████████| 3448/3448 [02:06<00:00, 27.34it/s]


In [374]:
Months = list()
for month in months:
    Months.append(month.upper())
Months[5]=Months[5].capitalize()
Months

['JANUARY',
 'FEBRUARY',
 'MARCH',
 'APRIL',
 'MAY',
 'June',
 'JULY',
 'AUGUST',
 'SEPTEMBER',
 'OCTOBER',
 'NOVEMBER',
 'DECEMBER']

In [375]:
df4 = df_s.copy().iloc[:,:2]
df4['Text'] = ''
df4['months'] = Months
df4

,Volume,Number,Text,months
0,1,No. 1 (January),,JANUARY
1,1,No. 2 (February),,FEBRUARY
2,1,No. 3 (March),,MARCH
3,1,No. 4 (April),,APRIL
4,1,No. 5 (May),,MAY
5,1,No. 6 (June),,June
6,1,No. 7 (July),,JULY
7,1,No. 8 (August),,AUGUST
8,1,No. 9 (September),,SEPTEMBER
9,1,No. 10 (October),,OCTOBER


In [376]:
slct = 'p.MsoNormal'
ttl = '' # 초기값
para = list() # 초기값
        
# 텍스트 분류
for text in tqdm(raw):
    if (text.replace(',','').split()[0] in Months)|(text.replace(',','').split()[0]=='June'):
        txt = '\n '.join(para)
        df4.loc[(df4['months']==ttl),'Text'] = txt
        ttl = text.replace(',','').split()[0]
        del txt
        para = list()
    para.append(text)
df4.loc[(df4['months']==ttl),'Text'] = '\n '.join(para)

100%|███████████████████████████████████████████████████████████████████████████| 2175/2175 [00:00<00:00, 32551.11it/s]


In [401]:
# 페이지 표시 제거
for i in tqdm(range(len(df4))):
    for j in range(0,1000):
        df4['Text'][i] = df4['Text'][i].replace('[page {}]'.format(j+1),'')

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]<ipython-input-401-406e7d5a9906>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Text'][i] = df4['Text'][i].replace('[page {}]'.format(j+1),'')
100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:07<00:00,  1.54it/s]


In [402]:
df4['Volume']=4
df4

,Volume,Number,Text,months
0,4,No. 1 (January),JANUARY 1904.\n A Point of Ethics.\n Chai Che-...,JANUARY
1,4,No. 2 (February),FEBRUARY 1904.\n The Russo-Japanese War.\n The...,FEBRUARY
2,4,No. 3 (March),"MARCH, 1904.\n The Russo-Japanese War.\n As ca...",MARCH
3,4,No. 4 (April),"APRIL, 1904.\n The Russo-Japanese War.\n We ar...",APRIL
4,4,No. 5 (May),"MAY, 1904.\n The Russo-Japanese War,\n At our ...",MAY
5,4,No. 6 (June),"June, 1904.\n The Russo-Japanese War,\n With t...",June
6,4,No. 7 (July),"JULY, 1904,\n Japanese Industrial Projects in ...",JULY
7,4,No. 8 (August),"AUGUST, 1904.\n The Fusion of Korean Society.\...",AUGUST
8,4,No. 9 (September),"SEPTEMBER, 1904.\n Spelling Reform.\n The foll...",SEPTEMBER
9,4,No. 10 (October),"OCTOBER, 1904.\n Koreans in Manchuria.\n Seven...",OCTOBER


### 5) vol 5. (1905)

In [425]:
soup = shtml(url_target+post[4])

In [426]:
slct = 'p.PreformattedText'
starting = textpoint(soup, 2, slct)

# 텍스트 저장
raw = list()
for i in tqdm(range(starting,len(soup.select(slct)))):
    _ = re.sub('\s+',' ',soup.select(slct)[i].text).strip()
    if _ != '':
        raw.append(re.sub('\s+',' ',soup.select(slct)[i].text).strip())

100%|██████████████████████████████████████████████████████████████████████████████| 4598/4598 [04:45<00:00, 16.12it/s]


In [427]:
df5 = df_s.copy().iloc[:,:2]
df5['Text'] = ''
df5['months'] = months
df5['Volume'] = 5
df5

,Volume,Number,Text,months
0,5,No. 1 (January),,January
1,5,No. 2 (February),,February
2,5,No. 3 (March),,March
3,5,No. 4 (April),,April
4,5,No. 5 (May),,May
5,5,No. 6 (June),,June
6,5,No. 7 (July),,July
7,5,No. 8 (August),,August
8,5,No. 9 (September),,September
9,5,No. 10 (October),,October


In [428]:
raw[0:10]

['THE KOREA REVIEW',
 'Vol. 5. No. 1. January, 1905.',
 'CONTENTS.',
 'Korea and Formosa 1',
 'The Iron Mines of Kang-wun Province 8',
 'The Russo-Japanese Conflict 12',
 'The Seoul Fusan Railway 16',
 'Odds and Ends',
 'Room at the top 21',
 'Society for the prevention of cruelty to animals 22']

In [429]:
list(df5.Number)

['No. 1 (January)',
 'No. 2 (February)',
 'No. 3 (March)',
 'No. 4 (April)',
 'No. 5 (May)',
 'No. 6 (June)',
 'No. 7 (July)',
 'No. 8 (August)',
 'No. 9 (September)',
 'No. 10 (October)',
 'No. 11 (November)',
 'No. 12 (December)']

In [430]:
slct = 'p.PreformattedText'
ttl = '' # 초기값
para = list() # 초기값
n = -1 # 초기값
        
# 텍스트 분류
for text in tqdm(raw):
    if text.startswith('THE KOREA REVIEW') is True:
        txt = '\n '.join(para)
        df5.loc[(df5.index==n),'Text'] = txt
        ttl = text.replace(',','').split()[0]
        del txt
        para = list()
        n+=1
    para.append(text)
df5.loc[(df5.index==n),'Text'] = '\n '.join(para)

# 페이지 표시 제거
for i in tqdm(range(len(df5))):
    for j in range(0,1000):
        df5['Text'][i] = df5['Text'][i].replace('[page {}]'.format(j+1),'')

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]<ipython-input-430-97ba35ab47d7>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Text'][i] = df5['Text'][i].replace('[page {}]'.format(j+1),'')
100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:07<00:00,  1.58it/s]


In [431]:
df5

,Volume,Number,Text,months
0,5,No. 1 (January),"THE KOREA REVIEW\n Vol. 5. No. 1. January, 190...",January
1,5,No. 2 (February),THE KOREA REVIEW\n VOL. 5. NO. 2. FEBRUARY\n C...,February
2,5,No. 3 (March),"THE KOREA REVIEW\n VOL. 5. NO. 3. MARCH, 1905\...",March
3,5,No. 4 (April),"THE KOREA REVIEW\n VOL. 5. NO. 4. APRIL, 1905....",April
4,5,No. 5 (May),"THE KOREA REVIEW,\n Vol. 5, No. 5, MAY 1905.\n...",May
5,5,No. 6 (June),THE KOREA REVIEW\n VOL. 5. NO. 6. JUNE. 1905.\...,June
6,5,No. 7 (July),"THE KOREA REVIEW\n VOL. 5. NO. 7. JULY, 1905.\...",July
7,5,No. 8 (August),"THE KOREA REVIEW\n VOL. 5. NO. 8. AUGUST, 1905...",August
8,5,No. 9 (September),"THE KOREA REVIEW\n VOL. 5. NO. 9. SEPTEMBER, 1...",September
9,5,No. 10 (October),"THE KOREA REVIEW.\n VOL.5. NO. l0. OCTOBER, 19...",October


### 6) vol 6. (1906)

In [432]:
soup = shtml(url_target+post[5])

In [433]:
slct = 'p.MsoNormal'
starting = textpoint(soup, 1, slct)

# 텍스트 저장
raw = list()
for i in tqdm(range(starting,len(soup.select(slct)))):
    _ = re.sub('\s+',' ',soup.select(slct)[i].text).strip()
    if _ != '':
        raw.append(re.sub('\s+',' ',soup.select(slct)[i].text).strip())

100%|██████████████████████████████████████████████████████████████████████████████| 3194/3194 [01:29<00:00, 35.76it/s]


In [434]:
df6 = df_s.copy().iloc[:,:2]
df6['Volume'] = 6
df6['Text'] = ''
df6

,Volume,Number,Text
0,6,No. 1 (January),
1,6,No. 2 (February),
2,6,No. 3 (March),
3,6,No. 4 (April),
4,6,No. 5 (May),
5,6,No. 6 (June),
6,6,No. 7 (July),
7,6,No. 8 (August),
8,6,No. 9 (September),
9,6,No. 10 (October),


In [435]:
slct = 'p.slct'
ttl = '' # 초기값
para = list() # 초기값
n = -1 # 초기값
        
# 텍스트 분류
for text in tqdm(raw):
    if text.startswith('THE KOREA REVIEW') is True:
        txt = '\n '.join(para)
        df6.loc[(df5.index==n),'Text'] = txt
        ttl = text.replace(',','').split()[0]
        del txt
        para = list()
        n+=1
    para.append(text)
df6.loc[(df6.index==n),'Text'] = '\n '.join(para)

# 페이지 표시 제거
for i in tqdm(range(len(df6))):
    for j in range(0,1000):
        df6['Text'][i] = df6['Text'][i].replace('[page {}]'.format(j+1),'')

  0%|                                                                                           | 0/12 [00:00<?, ?it/s]<ipython-input-435-f205468b4afe>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['Text'][i] = df6['Text'][i].replace('[page {}]'.format(j+1),'')
100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:07<00:00,  1.57it/s]


In [436]:
df6

,Volume,Number,Text
0,6,No. 1 (January),THE KOREA REVIEW.\n JANUARY. 1906.\n \n The Se...
1,6,No. 2 (February),THE KOREA REVIEW\n FEBRUARY 1906.\n \n Are the...
2,6,No. 3 (March),"THE KOREA REVIEW.\n MARCH, 1906\n [81]\n At Ki..."
3,6,No. 4 (April),"THE KOREA REVIEW\n APRIL, 1906.\n [121]\n Are ..."
4,6,No. 5 (May),"THE KOREA REVIEW\n May, 1906.\n [161]\n Gleani..."
5,6,No. 6 (June),THE KOREA REVIEW\n JUNE 1906.\n [201]\n Korean...
6,6,No. 7 (July),"THE KOREA REVIEW\n JULY, 1906.\n [241]\n The K..."
7,6,No. 8 (August),"THE KOREA REVIEW\n August, 1906\n [281]\n Ul-l..."
8,6,No. 9 (September),"THE KOREA REVIEW.\n [321]\n SEPTEMBER, 1906.\n..."
9,6,No. 10 (October),"THE KOREA REVIEW\n [361]\n OCTOBER, 1906 .\n M..."


## 3. 각 권(Vol) 합치기

In [450]:
df1 = df1.iloc[:,:-1]
df2 = df2.iloc[:,:-1]
df3 = df3.iloc[:,:-1]
df4 = df4.iloc[:,:-1]
df5 = df5.iloc[:,:-1]
df6

,Volume,Number,Text
0,6,No. 1 (January),THE KOREA REVIEW.\n JANUARY. 1906.\n \n The Se...
1,6,No. 2 (February),THE KOREA REVIEW\n FEBRUARY 1906.\n \n Are the...
2,6,No. 3 (March),"THE KOREA REVIEW.\n MARCH, 1906\n [81]\n At Ki..."
3,6,No. 4 (April),"THE KOREA REVIEW\n APRIL, 1906.\n [121]\n Are ..."
4,6,No. 5 (May),"THE KOREA REVIEW\n May, 1906.\n [161]\n Gleani..."
5,6,No. 6 (June),THE KOREA REVIEW\n JUNE 1906.\n [201]\n Korean...
6,6,No. 7 (July),"THE KOREA REVIEW\n JULY, 1906.\n [241]\n The K..."
7,6,No. 8 (August),"THE KOREA REVIEW\n August, 1906\n [281]\n Ul-l..."
8,6,No. 9 (September),"THE KOREA REVIEW.\n [321]\n SEPTEMBER, 1906.\n..."
9,6,No. 10 (October),"THE KOREA REVIEW\n [361]\n OCTOBER, 1906 .\n M..."


In [462]:
con_df = pd.concat([df1,df2,df3,df4,df5,df6]).reset_index().drop(columns=['index'])

## 4. 파일로 저장

### 1) csv로 저장

In [479]:
con_df.to_csv('korea_review.csv', encoding='utf-8', index=False)

### 2) json으로 저장

In [478]:
con_df.to_json('korea_review.json', orient='records')